<a href="https://colab.research.google.com/github/Hendrix-Nathan-Sumel/CPE-201L-DSA-2-A/blob/main/DSA%20NO.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Stack:
    def __init__(self):
        self.stack = []

    def add(self, dataval):
        self.stack.append(dataval)
        return True

    def remove(self):
        if len(self.stack) <= 0:
            return ("no element in the stack")
        else:
            return self.stack.pop

    def print_item(self):
        for item in self.stack:
            print(item,end='|')

Astack = Stack()
Astack.add("M")
Astack.add("A")
Astack.add("R")
Astack.add("K")
Astack.add(" ")
Astack.add("D")
Astack.add("A")
Astack.add("N")
Astack.add("I")
Astack.add("E")
Astack.add("L")
Astack.add("L")
Astack.add("E")
Astack.add(" ")
Astack.add("L")
Astack.add("A")
Astack.add("P")
Astack.add("U")
Astack.add("T")
Astack.print_item()

M|A|R|K| |D|A|N|I|E|L|L|E| |L|A|P|U|T|

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class QueueApp:
    def __init__(self):
        self.queue = []

        self.queue_output = widgets.Output()
        self.enqueue_button = widgets.Button(description="Enqueue")
        self.dequeue_button = widgets.Button(description="Dequeue")
        self.peek_button = widgets.Button(description="Peek")
        self.size_button = widgets.Button(description="Queue Size")
        self.display_button = widgets.Button(description="Display Queue")

        self.input_field = widgets.IntText(description="Item:")

        self.enqueue_button.on_click(self.enqueue)
        self.dequeue_button.on_click(self.dequeue)
        self.peek_button.on_click(self.peek)
        self.size_button.on_click(self.queue_size)
        self.display_button.on_click(self.display_queue)

        display(self.input_field, self.enqueue_button, self.dequeue_button,
                self.peek_button, self.size_button, self.display_button, self.queue_output)

    def enqueue(self, b):
        item = self.input_field.value
        self.queue.append(item)
        with self.queue_output:
            clear_output(wait=True)
            print(f"Item {item} added to the queue.")
            self.display_current_state()

    def dequeue(self, b):
        if self.queue:
            item = self.queue.pop(0)
            with self.queue_output:
                clear_output(wait=True)
                print(f"Item {item} removed from the queue.")
                self.display_current_state()
        else:
            with self.queue_output:
                clear_output(wait=True)
                print("Queue is empty! Cannot dequeue.")

    def peek(self, b):
        if self.queue:
            item = self.queue[0]
            with self.queue_output:
                clear_output(wait=True)
                print(f"Front item: {item}")
        else:
            with self.queue_output:
                clear_output(wait=True)
                print("Queue is empty!")

    def queue_size(self, b):
        with self.queue_output:
            clear_output(wait=True)
            print(f"Queue size: {len(self.queue)}")

    def display_queue(self, b):
        with self.queue_output:
            clear_output(wait=True)
            self.display_current_state()

    def display_current_state(self):
        print("Current Queue:", self.queue)


# Run the QueueApp
queue_app = QueueApp()


IntText(value=0, description='Item:')

Button(description='Enqueue', style=ButtonStyle())

Button(description='Dequeue', style=ButtonStyle())

Button(description='Peek', style=ButtonStyle())

Button(description='Queue Size', style=ButtonStyle())

Button(description='Display Queue', style=ButtonStyle())

Output()

In [1]:
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output
import hashlib

# Connect to SQLite Database
def connect_db():
    conn = sqlite3.connect('shop_queue.db')
    return conn

# Create Tables if they don't exist
def create_tables():
    conn = connect_db()
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT UNIQUE,
                    password TEXT,
                    role TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS orders (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER,
                    order_name TEXT,
                    status TEXT,
                    FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit()
    conn.close()

# Hash Password (for security)
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Admin account check
def check_admin():
    conn = connect_db()
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE role = 'admin'")
    admin = c.fetchone()
    conn.close()
    return admin

# Create an Account (User or Admin)
def create_account(username, password, role='user'):
    conn = connect_db()
    c = conn.cursor()
    hashed_password = hash_password(password)
    try:
        c.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
                  (username, hashed_password, role))
        conn.commit()
        conn.close()
        return True
    except sqlite3.IntegrityError:
        conn.close()
        return False

# Authenticate User
def authenticate(username, password):
    conn = connect_db()
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE username = ?", (username,))
    user = c.fetchone()
    conn.close()
    if user and hash_password(password) == user[2]:
        return user
    return None

# Add a New Order
def add_order(user_id, order_name):
    conn = connect_db()
    c = conn.cursor()
    c.execute("INSERT INTO orders (user_id, order_name, status) VALUES (?, ?, ?)",
              (user_id, order_name, 'Pending'))
    conn.commit()
    conn.close()

# Get Orders of a User
def get_user_orders(user_id):
    conn = connect_db()
    c = conn.cursor()
    c.execute("SELECT * FROM orders WHERE user_id = ? ORDER BY id DESC", (user_id,))
    orders = c.fetchall()
    conn.close()
    return orders

# Get All Orders (Admin Access)
def get_all_orders():
    conn = connect_db()
    c = conn.cursor()
    c.execute("SELECT users.username, orders.order_name, orders.status FROM orders JOIN users ON orders.user_id = users.id")
    orders = c.fetchall()
    conn.close()
    return orders

# Update Order Status (Admin can process)
def process_order(order_id):
    conn = connect_db()
    c = conn.cursor()
    c.execute("UPDATE orders SET status = 'Processed' WHERE id = ?", (order_id,))
    conn.commit()
    conn.close()

# Update Username and Password
def update_account(user_id, new_username, new_password):
    conn = connect_db()
    c = conn.cursor()
    hashed_password = hash_password(new_password)
    c.execute("UPDATE users SET username = ?, password = ? WHERE id = ?",
              (new_username, hashed_password, user_id))
    conn.commit()
    conn.close()

# Main Application Class
class ComputerShopQueueApp:
    def __init__(self):
        self.current_user = None
        self.ui_colors = {'background': 'white', 'text': 'black'}  # Default UI colors
        create_tables()

        self.login_widgets()

    def login_widgets(self):
        self.username_input = widgets.Text(description="Username:")
        self.password_input = widgets.Password(description="Password:")
        self.login_button = widgets.Button(description="Login")
        self.register_button = widgets.Button(description="Create Account")

        self.login_button.on_click(self.login)
        self.register_button.on_click(self.show_registration_form)

        display(self.username_input, self.password_input, self.login_button, self.register_button)

    def login(self, b):
        username = self.username_input.value
        password = self.password_input.value
        user = authenticate(username, password)

        if user:
            self.current_user = user
            with self.username_input:
                clear_output(wait=True)
                print(f"Welcome, {user[1]}!")
                if user[3] == 'admin':
                    self.admin_dashboard()
                else:
                    self.customer_dashboard()
        else:
            with self.username_input:
                clear_output(wait=True)
                print("Invalid username or password.")

    def show_registration_form(self, b):
        self.username_input.value = ""
        self.password_input.value = ""
        self.login_button.description = "Register"
        self.login_button.on_click(self.register_account)

    def register_account(self, b):
        username = self.username_input.value
        password = self.password_input.value
        if create_account(username, password, role='user'):
            with self.username_input:
                clear_output(wait=True)
                print(f"Account created for {username}. Please login.")
            self.login_widgets()
        else:
            with self.username_input:
                clear_output(wait=True)
                print("Username already exists.")

    def admin_dashboard(self):
        self.admin_widgets()

    def admin_widgets(self):
        self.admin_output = widgets.Output()
        self.view_orders_button = widgets.Button(description="View All Orders")
        self.process_order_button = widgets.Button(description="Process Order")
        self.settings_button = widgets.Button(description="Settings")

        self.view_orders_button.on_click(self.view_all_orders)
        self.process_order_button.on_click(self.process_order)
        self.settings_button.on_click(self.open_settings)

        display(self.view_orders_button, self.process_order_button, self.settings_button, self.admin_output)

    def customer_dashboard(self):
        self.customer_widgets()

    def customer_widgets(self):
        self.customer_output = widgets.Output()
        self.order_input = widgets.Text(description="Order:")
        self.submit_order_button = widgets.Button(description="Submit Order")
        self.submit_order_button.on_click(self.submit_order)

        display(self.order_input, self.submit_order_button, self.customer_output)

    def submit_order(self, b):
        if self.current_user:
            add_order(self.current_user[0], self.order_input.value)
            with self.customer_output:
                clear_output(wait=True)
                print("Order submitted.")

    def view_all_orders(self, b):
        orders = get_all_orders()
        with self.admin_output:
            clear_output(wait=True)
            print("All Orders:")
            for order in orders:
                print(f"Customer: {order[0]}, Order: {order[1]}, Status: {order[2]}")

    def process_order(self, b):
        order_id = int(input("Enter the order ID to process: "))
        process_order(order_id)
        with self.admin_output:
            clear_output(wait=True)
            print("Order processed successfully.")

    def open_settings(self, b):
        self.settings_widgets()

    def settings_widgets(self):
        self.settings_output = widgets.Output()

        # Username and Password Change
        self.new_username_input = widgets.Text(description="New Username:")
        self.new_password_input = widgets.Password(description="New Password:")
        self.change_account_button = widgets.Button(description="Change Account Info")
        self.change_account_button.on_click(self.change_account_info)

        # UI Customization
        self.bg_color_input = widgets.ColorPicker(value=self.ui_colors['background'], description='Background Color:')
        self.text_color_input = widgets.ColorPicker(value=self.ui_colors['text'], description='Text Color:')
        self.apply_ui_button = widgets.Button(description="Apply UI Changes")
        self.apply_ui_button.on_click(self.apply_ui_changes)

        display(self.new_username_input, self.new_password_input, self.change_account_button,
                self.bg_color_input, self.text_color_input, self.apply_ui_button, self.settings_output)

    def change_account_info(self, b):
        new_username = self.new_username_input.value
        new_password = self.new_password_input.value
        if new_username and new_password:
            update_account(self.current_user[0], new_username, new_password)
            with self.settings_output:
                clear_output(wait=True)
                print("Account info updated successfully.")
                self.current_user = authenticate(new_username, new_password)  # Update the current user details
        else:
            with self.settings_output:
                clear_output(wait=True)
                print("Please enter a new username and password.")

    def apply_ui_changes(self, b):
        # Apply background and text color changes
        self.ui_colors['background'] = self.bg_color_input.value
        self.ui_colors['text'] = self.text_color_input.value
        display(f"Applied new UI settings: Background - {self.ui_colors['background']}, Text - {self.ui_colors['text']}")

        # Update the UI display colors dynamically
        display(widgets.HTML(f"<div style='background-color: {self.ui_colors['background']}; color: {self.ui_colors['text']}; padding: 20px;'>UI Customization Applied</div>"))

# Run the ComputerShopQueueApp
app = ComputerShopQueueApp()

Text(value='', description='Username:')

Password(description='Password:')

Button(description='Login', style=ButtonStyle())

Button(description='Create Account', style=ButtonStyle())